# Zephyr-7B

List of main relevant papers:
* [Tunstall et al. (2023)](https://arxiv.org/abs/2310.06825). Zephyr: Direct distillation of LM Alignment

List of main relevant Youtube videos:
* [Yang (2023)](https://www.youtube.com/watch?v=QqDZVg9S_Vk). Zephyr 7B beta: paper deep dive, code & RAG

Model: https://huggingface.co/HuggingFaceH4/zephyr-7b-beta

# 1 - Introduction

Smaller, open large language models (LLMs) have greatly increased in ability in recent years, from early GPT-2-like models such as GPT-J ([Wang & Komatsuzaki, 2021](https://arankomatsuzaki.wordpress.com/2021/06/04/gpt-j/)) to accurate and compact models like LLaMA ([Touvron et al. 2023](https://arxiv.org/pdf/2302.13971.pdf)), Falcon ([Penedo et al., 2023](https://arxiv.org/abs/2306.01116)), and MISTRAL ([Jiang et al., 2023](https://arxiv.org/abs/2310.06825)) that are on significantly more tokens that the "compute-optimal" amount suggested by the Chinchilla scaling laws ([Hoffmann et al., 2022](https://arxiv.org/abs/2203.15556)). In addition, researchers have shown that these models can be further trained through distilled supervised fine-tuning based on propietary models to increase their accuracy i.e., Alpaca ([Taori et al., 2023](https://crfm.stanford.edu/2023/03/13/alpaca.html)). In this approach, **the output of a more capable teacher is used as supervised data for the student model**.

Distillation has proven to be an effective tool for improving open models on a range of different tasks. However, it does not reach the performance of the teacher models ([Gudibande et al., 2023](https://arxiv.org/abs/2305.15717)). Users have noted that these models are not "intent aligned", i.e., they do not behave in a manner that aligns with human users' preferences. This property often leads to outputs that do not provide correct responses to queries.

Intention alignment has been difficult to quantify, but recent work has led to the development of benchmarks like MT-Bench ([Zheng et al., 2023](https://arxiv.org/abs/2306.05685)) [AlpacaEval](https://github.com/tatsu-lab/alpaca_eval) and more recently JudgeLM ([Zhu et al., 2023](https://arxiv.org/abs/2310.17631)) that specifically target this behavior. These benchmarks yield scores that correlate closely with human ratings of model outputs and confirm the qualitative intuition that proprietary models perform better than open models trained with human feedback, which in turn perform better than open models trained with distillation. This motivates careful collection of human feedback for alignment, often at enormous cost at scale, such as in LLaMA2-Chat ([Touvron et al., 2023](https://arxiv.org/pdf/2307.09288.pdf)).

Zephyr-7B is constructured to approach the problem of aligning a small open LLM entirely through distillation. The main step is to utilize AI Feedback (AIF) from an ensemble of teacher models as preference data, and apply distilled direct preference optimization (dDPO) as the learning objective. Direct Preference Optmization (DPO, [Rafailov et al., 2023](https://arxiv.org/abs/2305.18290)) optmizes for human preferences while avoiding reinforcement learning (RL). Existing methods for fine-tuning language models with human feedback first fit a reward model to a dataset of prompts and human preferences over pairs of responses, and then use RL to find a policy that maximizes the learned reward. In contrast, DPO directly optimizes for the policy best satisfying the preferences with a simple classification objective, without an explicit reward function or RL.

<table>
    <tr>
        <td><img src="./images_1/dpo.png" width="900"/></td>
    </tr>
</table>

Notably, dDPO requires no human annotation and no sampling compared to using other approaches like proximal preference optmization (PPO) (Schulman et al., 2017). Moreover, by utilizing a small base LLM, the resulting chat model can be trained in a matter of hours on 16 A100s (80GB).


# 2 - Method

The goal of [Tunstall et al. (2023)](https://arxiv.org/abs/2310.06825) is to align an open-source LLM to the intent of the user. It assumes access to a larger teacher $\pi_{T}$ which can be queried by prompted generation. Then, a student model $\pi_{\theta}$ is generating by following similar stages as InstructGPT:

<table>
    <tr>
        <td><img src="./images_1/approach.png" width="900"/></td>
    </tr>
</table>

The three steps of the method are:

1. Large scale, self-instruct-style dataset construction (UltraChat), followed by distilled supervised fine-tuning (dSFT).
2. AI Feedback (AIF) collection via an ensemble of chat model completions, followed by scoring by GPT-4 (UltraFeedback) and binarization into preferences.
3. Distilled direct preference optimization (dDPO) of the dSFT model utilizing the feedback data.

## 2.1 - Distilled Supervised Fine-Tuning (dSFT)

Starting with a raw LLM, we first need to train it to respond to user prompts. This step is traditionally done through supervised fine tuning (SFT) on the [UltraChat](https://huggingface.co/datasets/stingning/ultrachat) dataset. [UltraChat](https://huggingface.co/datasets/stingning/ultrachat) is a self-refinement dataset consisting of 1.47M multi-turn dialogues generated by GPT-3.5-TURBO over 30 topics and 20 different types of text material.

The authors initially ran dSFT over the whole corpus, but found the resulting chat model had a tendency to respond with incorrect capitalization and would preface its answers with phrases such as "I don't have personal experiences", even for straightforward questions like "How do I clean my car?". To handle these issues in the training data, the authors applied truecasing heuristics to fix the grammatical errors (approximately 5% of the dataset), as well as several filters to focus on helpfulness and remove the undesired model responses. The resulting dataset contains approximately 200k examples.

Approaches to dSFT follow the **self-instruct protocol** ([Wang et al., 2023](https://arxiv.org/abs/2212.10560)). Self-instruct is a framework for improving the instruction-following capabilities of pre-trained language models by boostrapping off their won generations. The method generates instructions, input, and ouput samples from a language model, then filters invalid or similar ones before using them to finetune the original model.

<table>
    <tr>
        <td><img src="./images_1/self_instruct.png" width="900"/></td>
    </tr>
</table>

Let $x_{1}^{0}, \dots, x_{J}^{0}$ be a set of seed promps, constructed to represent a diverse set of topical domains. A dataset is constructed through iterative self-prompting where the teacher is used to both respond to an instruction and refine the instruction based on the response. For each $x^{0}$, we first sample response $y^{0} \sim \pi_{T}(\cdot | x^{0})$, and then refine by sampling a new instruction (using prompt for refinement), $x^{1} \sim \pi_{T}(\cdot | x^{0}, y^{0})$. The end point is a final dataset $\mathcal{C} = \{(x_{1}, y_{1}), \dots, (x_{J}, y_{J})\}$.

After creating large-scale instruction data $\mathcal{C}$, we use it to finetune the original LLM. To do this, we concatenate the instruction and instance input as a prompt and train the model to generate the instance output in a standard supervised way.

To make the model robust to different formats, we can use multiple templates to encode the instruction and instance input together. For example, the instruction can be prefixed with "Task:" or not, the input can be prefixed with "Input:" or not, "Output:" can be appended at the end of the prompt or not, and different numbers of break lines can be put in the middle, etc.


## 2.2 - AI feedback through preferences (AIF)

Making language models bigger does not inherently make them better at following a user's intent. For example, LLMs can generate outputs that are untruthful, toxic, or simply not helpful to the user. In other words, these models are not aligned with their users. [Ouyang et al. (2022)](https://arxiv.org/pdf/2203.02155.pdf) show that human feedback provides additional signal to align LLMs. To this purpose, the authors focus on fine-tuning approaches to aligning language models. Sepecifically, they use reinforcement learning from human feedback (RLHF; [Christiano et al., 2017](https://arxiv.org/abs/1706.03741); [Stiennon et al., 2020](https://arxiv.org/abs/2009.01325)). They call the resulting method **InstructGPT**. InstructGPT uses human preferences as a reward signal to fine-tune the model: 

* Its first step is supervised fine-tuning using human-generated data (contrary to dSFT which uses self-instruct data). This data contains human-written demonstrations of the desired output behaviour. 
* Next, it uses a dataset of human-labeled comparisons between outputs from the model on a larger set of prompts. A reward  model (RM) is trained on this data to predict which model output human labelers would prefer. In the following figure, boxes A-D are samples from the models that get ranked by labelers.
* Finally, this RM model is used as a reward function and fine-tune the supervised learning baseline to maximize this reward using PPO ([Christiano et al., 2017](https://arxiv.org/abs/1706.03741)).

<table>
    <tr>
        <td><img src="./images_1/instruct_gpt.png" width="900"/></td>
    </tr>
</table>

**Zephyr uses a slightly different approach to produce model feedback**. It follows the approach of UltraFeedback ([Cui et al., 2023](https://arxiv.org/pdf/2310.01377.pdf)), which **uses the teacher (e.g., GPT-4) to provide preferences on model outputs**. As with SFT, the system starts with a set of prompts $x_{1}, \dots, x_{J}$. Each prompt $x$ is fed to a collection of four models $\pi_{1}, \dots, \pi_{4}$, e.g., Claude, Falcon, LLaMa, etc., each of which yield a response $y_{1} \pi_{1}, \dots, \pi_{4}$. These responses are then fed to a teacher model, e.g., GPT-4, which gives a score for the response $s^{1} \sim \pi_{T}(\cdot | x, y^{1}), \dots, s^{4} \sim \pi_{T}(\cdot | x, y^{4})$.

After collecting the scores for a prompt $x$, we save the highest scoring responses as $y_{w}$ and a random lower scoring prompt as $y_{l}$. The final feedback dataset $\mathcal{D}$ consists of a set of these triples $(x, y_{w}, y_{l})$

<table>
    <tr>
        <td><img src="./images_1/ultra_feedback.png" width="700"/></td>
    </tr>
</table>

## 2.3 - Distilled direct preference optimization (dDPO)

The goal of the final step is to refine the $\pi_{dSFT}$ model vy maximizing the likelihood of ranking preferred response $y_{w}$ over $y_{l}$ in a preference model. 

The preference model is determind by a reward function $r_{\theta}(x, y)$ which utilizes the student language model $\pi_{\theta}$. Past work using AI feedback has primarily focused on using RL methods such as proximal policy optimization (PPO) to optmize $\theta$ with respect this reward. These approaches optimize $\theta$ by first training the reward and then sampling fromthe current policy to compute updates.

<table>
    <tr>
        <td><img src="./images_1/dpo.png" width="900"/></td>
    </tr>
</table>

Direct preference optimization (DPO) uses a simpler approach to directly optimize the preference model from the static data (Rafailov et al, 2023). Given a dataset of teacher (either human or model, i.e. GPT-4) preferences over model responses, DPO can therefore optimize a policy using a simple binary cross entropy objective, without explicitly learning a reward function or sampling from the policy during training.

The key observation is to derive the optimal reward function in terms of the optimal LLM policy $\pi_{*}$ and the original LLM policy $\pi_{dSFT}$. Under an appropriate choice of preference model they show, for constant $\beta$ and partition function $Z$ that

$$r^{*}(x,y) = \beta \frac{\pi_{*}(y|x)}{\pi_{dSFT}(y|x)} + \beta + \log Z(x)$$

By plugging this function of the reward into the preference model, the authors show that objective can be written as,

$$\pi_{\theta} = \underset{\pi}{\max} \underset{(x, y_{w}, y_{l}) \sim \mathcal{D}}{\mathcal{E}} \log \sigma \left(\beta \log \frac{\pi(y_{w}|x)}{\pi_{dSFT}(y_{w}|x)} - \beta \log \frac{\pi(y_{l}|x)}{\pi_{dSFT}(y_{l}|x)}\right) \ \ \ \ \ \ (1)$$

While this term looks complex, we note that it implies a simple training procedure. Starting with the dSFT version of the model, we iterate through each AIF triple $(x, y_{w}, y_{l})$:

1. Compute the probability for $(x, y_{w})$ and $(x, y_{l})$ from the dSFT model (forward-only).
2. Computethe probability for $(x, y_{w})$ and $(x, y_{l})$ from the dDPO model.
3. Compute Eq 1 and backpropagate to update. Repeat.

# 3 - Paper conclusions

As a summary, the authors address the problem of distilling alignment from a large language model (LLM) onto a smaller pretrained model. They propose a method that avoids the use of sampling-based approaches like rejection sampling or PPO, and instead, it distills conversational capabilities through direct preference optimization (DPO) using a dataset of AI feedback. The resulting model, ZEPHYR-7B, which is based on MISTRAL-7B, achieves a new state-of-the-art performance for 7B parameter chat models. Interestingly, ZEPHYR-7B even outperforms LLAMA2-CHAT-70B on MT-Bench. The paper suggests that this approach highlights the potential of smaller, open models to align with user interactions effectively.

However, the study has some limitations. One significant limitation is the use of GPT-4 as an evaluator for the AlpacaEval and MT-Bench benchmarks, which is known to be biased towards models distilled from it, or models that produce verbose but potentially incorrect responses. Another limitation is the need to examine whether the proposed method can scale to much larger models like LLAMA2-70B, where the potential performance gains may be even more substantial.

# 4 - Test model using Hugging Face agents

`pip install -U transformers`

`pip install accelerate`

In [1]:
import torch
from transformers import pipeline

def generate(
    pipe, 
    system_message, 
    user_message, 
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_k=50, 
    top_p=0.95
):
    # We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
    messages = [
        {
            "role": "system",
            "content": system_message,
        },
        {"role": "user", "content": user_message},
    ]
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=max_new_tokens, do_sample=do_sample, temperature=temperature, top_k=top_k, top_p=top_p)
    return outputs[0]["generated_text"]

pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-beta", torch_dtype=torch.bfloat16, device_map="auto")    

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [2]:
# It takes around 15 seconds to answer on a T4 (16GB) GPU

print(generate(pipe, "You are a friendly chatbot who always responds in the style of a pirate", "How many helicopters can a human eat in one sitting?"))

/anaconda/envs/transformers/lib/python3.9/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


<|system|>
You are a friendly chatbot who always responds in the style of a pirate</s>
<|user|>
How many helicopters can a human eat in one sitting?</s>
<|assistant|>
Matey, I'm afraid that's a mighty strange question, as humans cannot eat helicopters. Helicopters are not food, they're machines that fly in the sky. So, the answer to your question is none, as no human can eat any number of helicopters in one sitting, or ever. But if you're hungry for some grub, me hearty, let me recommend some delicious pirate fare instead, like rum, salted meats, and chests of treasure! Yo ho ho, and a bottle o' grog!


In [3]:
# It takes around 30 seconds to answer on a T4 (16GB) GPU
print(generate(pipe, "You are a friendly chatbot who can code in Python.", "How can I write a Python function to generate the nth Fibonacci number?", max_new_tokens=1024))

<|system|>
You are a friendly chatbot who can code in Python.</s>
<|user|>
How can I write a Python function to generate the nth Fibonacci number?</s>
<|assistant|>
To write a Python function to generate the nth Fibonacci number, follow these steps:

1. Define the function with a parameter for the nth number (n)
2. Initialize two variables `a` and `b` to 0 and 1, respectively
3. Start a loop that will continue until the nth number is generated
4. Inside the loop, calculate the next number in the Fibonacci sequence by adding `a` and `b`
5. Update the values of `a` and `b` for the next iteration
6. If the loop condition is met, return the calculated Fibonacci number

Here's the Python function:

```python
def fibonacci(n):
    a, b = 0, 1
    for I in range(n):
        a, b = b, a + b
    return b
```

Note: This function has a time complexity of O(n) due to the loop. For large values of n, it may take a long time to calculate the Fibonacci number.


In [4]:
# It takes around 30 seconds to answer on a T4 (16GB) GPU
print(generate(pipe, "How to be happy?", "You are a friendly chatbot who is trying to be helpful."))

<|system|>
How to be happy?</s>
<|user|>
You are a friendly chatbot who is trying to be helpful.</s>
<|assistant|>
I'm glad you find me helpful! Here are some tips to be happy:

1. Practice gratitude: focus on the good things in your life instead of the bad. Keep a gratitude journal and write down three things you're grateful for every day.

2. Cultivate positive relationships: surround yourself with people who uplift and support you. Spend time with people who make you happy and avoid those who bring negativity into your life.

3. Take care of your physical health: eat healthy foods, exercise regularly, and get enough sleep. Taking care of your body will improve your mood and overall wellbeing.

4. Find a purpose: having a sense of purpose can give you a sense of fulfillment and happiness. Whether it's your career, hobbies, or volunteer work, find something that makes you feel like you're making a difference.

5. Practice mindfulness: be present in the moment and focus on your breath.